In [1]:
import sys, os
from pathlib import Path
if '_old_cwd' not in locals():
    _old_cwd = os.getcwd()
    os.chdir(Path(os.getcwd()).parent)
    sys.path.append(os.getcwd())
# from ib_insync import *
# from tqdm.notebook import tqdm
from omnibelt import tqdmd_notebook as tqdm
from tabulate import tabulate
from omnibelt import save_json, load_json
from collections import Counter
import xmltodict

from src.euro import IB_Extractor


In [2]:
root = Path(os.getcwd()) / 'ibkr'

In [3]:
ibe = IB_Extractor(True)

ERROR:ib_insync.wrapper: Error 321, reqId 2147483647: Error validating request.-'cB' : cause - The API interface is currently in Read-Only mode.
ERROR:ib_insync.wrapper: Error 321, reqId -1: Error validating request.-'b_' : cause - The API interface is currently in Read-Only mode.
ERROR:ib_insync.ib: open orders request timed out
ERROR:ib_insync.ib: completed orders request timed out


In [4]:
raw = ['ASML', 'MC', 'TTE', 'SAP', 'SIE', 'SAN', 'OR', 'SU', 'ALV', 'AI', 'AIR', 'RMS', 'BNP', 'DTE', 'IBE', 'DG', 'SAN1', 'MBG', 'SAF', 'EL', 'CS', 'IFX', 'BAYN', 'PRX', 'ENEL', 'ADYEN', 'INGA', 'ABI', 'MUV2', 'DHL', 'UCG', 'STLAP', 'BBVA', 'BAS', 'ISP', 'RI', 'CRH', 'ITX', 'KER', 'NDA', 'BMW', 'BN', 'DB1', 'FLTR', 'ENI', 'ADS', 'AD', 'VOW3', 'NOKIA', 'VNA']
wts = [8.22, 6.88, 4.3, 4.22, 3.63, 3.47, 3.21, 2.89, 2.74, 2.67, 2.46, 2.22, 2.2, 2.15, 2.09, 1.97, 1.89, 1.82, 1.79, 1.73, 1.69, 1.63, 1.63, 1.57, 1.52, 1.51, 1.5, 1.46, 1.46, 1.44, 1.4, 1.37, 1.36, 1.36, 1.33, 1.27, 1.27, 1.21, 1.18, 1.16, 1.11, 1.11, 1.03, 1.0, 0.99, 0.96, 0.96, 0.78, 0.59, 0.45]

In [5]:
cts_data = {}

In [6]:
pbar = tqdm(raw)
for sym in pbar:
    if sym in cts_data:
        continue
    pbar.set_description(sym)
    cts_data[sym] = ibe.find_all_contracts(sym)

  0%|          | 0/50 [00:00<?, ?it/s]

In [16]:
def extract_contract_info(contract):
    return {'currency': contract.currency, 'exchange': contract.exchange, 'symbol': contract.symbol, 'secType': contract.secType, 'conId': contract.conId, 'primaryExchange': contract.primaryExchange}
def contract_from_info(info):
    return Contract(conId=info['conId'], symbol=info['symbol'], secType=info['secType'], currency=info['currency'], exchange=info['exchange'], primaryExchange=info['primaryExchange'])

In [8]:
cts_options = [[extract_contract_info(c) for c in cts_data[sym]] for sym in raw]
len(cts_options)

50

In [9]:
by_ID = {}
for k, vs in cts_data.items():
    for v in vs:
        if v.conId not in by_ID:
            by_ID[v.conId] = extract_contract_info(v)

In [10]:
# save_json(by_ID, root / 'all-stoxx-contracts.json');

In [11]:
by_ID = load_json(root / 'all-stoxx-contracts.json')
len(by_ID)

130

In [12]:
eur = [v for v in by_ID.values() if v['currency'] == 'EUR']
len(eur)

50

In [17]:
snaps = [ibe.snapshot(contract_from_info(e)) for e in tqdmd(eur, lambda v: v['symbol'])]

  0%|          | 0/50 [00:00<?, ?it/s]

In [19]:
snap_dicts = [xmltodict.parse(s) for e, s in zip(eur, snaps)]
len(snap_dicts)

50

In [22]:
# save_json(snap_dicts, root / 'all-stoxx-snapshots.json')

In [23]:
out = xmltodict.parse(ibe.recommendations(contract_from_info(eur[0])))